In [2]:
%load_ext autoreload
%autoreload 2
from expressiveness_benchmark.types import Plan, Task, Language, SourceRange, Program
from code_widget.example import CodeWidget
from dataclasses import replace
import json
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# CHANGE ME!
TASK_ID = 'tom_hanks'
AUTHOR = 'g'


In [4]:
task = Task(
    id=TASK_ID,
    name="Directors of movies Tom Hanks starred in",
    description="Find directors of movies that Tom Hanks starred in",
    plan=[
        {
        "id": "director",
        "description": "Find directors",
    }, {
        "id": "actor",
        "description": "Tom Hanks starred in",
    },
        {
        "id": "movies",
        "description": "of movies",
    } 
    ],
    sample_input={
        "actors":[
            {"movie":"AM","actor":"Tom Hanks"},
            {"movie":"AM","actor":"Not Tom Hanks"},
            {"movie":"AM","actor":"Also not Tom Hanks"},
            {"movie":"BM","actor":"Other actor"},
            {"movie":"CM","actor":"Other actor"},
        ],
        "directors":[
            {"director":"A","movie":"AM"},
            {"director":"A","movie":"BM"},
            {"director":"C","movie":"CM"},
        ],
    },
    sample_output=["A"],
    category='Joins'
)
task.save()

prototype = Program(
    task=TASK_ID,
    author=AUTHOR,
    language=''    
)

In [13]:
q = replace(prototype,
    language='q',
    source='''tom_hanks:
  (select director from actors ij (`movie xkey directors) 
   where actor ~\: "Tom Hanks") `director''').load_plan()
q.execute(task)
q.save()

In [14]:
q.widget(task)

Output()

CodeWidget(program='{"task": "tom_hanks", "language": "q", "plan": {}, "source": "tom_hanks:\\n  (select direc…

In [5]:
r = replace(prototype,
    language='r',
    source='''tom_hanks <- function(actors, directors) {
  actors %>%
    filter(actor == "Tom Hanks") %>%
    inner_join(directors, by = "movie") %>%
    pull(director)
}''').load_plan()
r.execute(task)
r.save()

In [6]:
r.widget(task)

Output()

CodeWidget(program='{"task": "tom_hanks", "language": "r", "plan": {}, "source": "tom_hanks <- function(actors…

In [5]:
imp = replace(prototype,
    language='python-imperative',
    source='''
def tom_hanks(actors,directors):
  r_directors = set()
  for a in actors:
    if 'Tom Hanks' == a['actor']:
      for d in directors:
        if a['movie'] == d['movie']:
          r_directors.add(d['director'])
    return r_directors
''').load_plan()
imp.execute(task)
imp.save()

In [6]:
func = replace(prototype,
    language='python-functional',
    source='''
def tom_hanks(actors,directors):
  movies = [a['movie'] for a in actors \
    if a['actor'] == 'Tom Hanks']
  directors = [d['director'] for d in directors \
    if d['movie'] in movies]
  return directors
''').load_plan()
func.execute(task)
func.save()

In [7]:
pandas = replace(prototype,
    language='python-pandas',
    source='''
def tom_hanks(actors,directors):
  movies = list(actors.loc[
    actors['actor'] == 'Tom Hanks', 'movie'])
  return directors[directors['movie'].
    isin(movies)]['director']
''').load_plan()
pandas.execute(task)
pandas.save()

In [8]:
sql = replace(prototype,
    language='sql',
    source='''
SELECT directors.director
FROM 
  directors 
  INNER JOIN actors
  ON directors.movie = 
  actors.movie
WHERE actors.actor="Tom Hanks"
''').load_plan()
sql.execute(task)
sql.save()

In [9]:
datalog = replace(prototype,
    language='datalog',
    source='''
tom_hanks(Director):-
  actors("Tom Hanks",movie),
  directors(Director,movie).
''').load_plan()
datalog.execute(task)
datalog.save()

In [14]:
program = datalog
program.widget(task)

Output()

CodeWidget(program='{"task": "tom_hanks", "language": "datalog", "plan": {"director": [{"line": 3, "start": 2,…